In [1]:
catName = 'low'

In [2]:
import sys, os, pickle, time, re
from glob import glob
sys.path.append('../lib')
sys.path.append('../analysis')

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from array import array

import ROOT as rt
rt.TGaxis.SetMaxDigits(4)
rt.gErrorIgnoreLevel = rt.kError
rt.RooMsgService.instance().setGlobalKillBelow(rt.RooFit.ERROR)
import ROOT.RooFit as rf
import root_numpy as rtnp

from analysis_utilities import drawOnCMSCanvas, getEff
from histo_utilities import create_TH1D, create_TH2D, SetMaxToMaxHist
from gridVarQ2Plot import col_dic, label_dic
from progressBar import ProgressBar
from categoriesDef import categories
cat = categories[catName]

from lumi_utilities import getLumiByTrigger

import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 1


CMS_lumi.extraText = 7*' ' + 'Internal'

donotdelete = []

Welcome to JupyROOT 6.12/07


In [4]:
m_mu = 0.1056583755
m_pi = 0.13957062
m_K = 0.493677
m_D0 = 1.86483
m_Dst = 2.01026
m_jpsi = 3.096916
dm = m_Dst - m_D0

In [5]:
catText = rt.TLatex()
catText.SetTextAlign(31)
catText.SetTextSize(0.06)
catText.SetTextFont(42)
catText.SetTextSize(0.05)

In [6]:
cl = rt.TLine()
cl.SetLineColor(6)
cl.SetLineStyle(9)
cl.SetLineWidth(2)

In [8]:
df = pd.DataFrame(rtnp.root2array('../data/cmsRD/skimmed/B2DstMu_200410_{}_corr.root'.format(cat.name)))

In [ ]:
loc = '../data/cmsMC_private/'
loc += 'BPH_Tag-B0_MuNuDmst-pD0bar-kp_13TeV-pythia8_Hardbbbar_PTFilter5_0p0-evtgen_ISGW2_PU20_10-2-3/'
loc += 'ntuples_B2DstMu/skimmed/{}.root'.format(cat.name)
df_MC = pd.DataFrame(rtnp.root2array(loc))

In [ ]:
'  '.join(df.columns)

In [ ]:
dataDir = '../data/cmsRD'
datasets_loc = glob(dataDir + '/ParkingBPH*/*2018*B2DstMu_200327*')
lumi_tot = getLumiByTrigger(datasets_loc, cat.trg, verbose=True)
CMS_lumi.integrated_lumi = lumi_tot

In [ ]:
fBfield = rt.TFile.Open('../data/calibration/bFieldMap_2Dover3D.root', 'r')
hBfield = fBfield.Get('bfieldMap')
def get_bFieldCorr3D(phi, eta, verbose=False):
    if not verbose:
        idx = hBfield.GetBin(hBfield.GetXaxis().FindBin(phi), hBfield.GetYaxis().FindBin(eta))
        return 1./hBfield.GetBinContent(idx)
hBfield.SetXTitle('#phi [rad]')
hBfield.SetYTitle('#eta')
c = rt.TCanvas('c', 'c', 800, 600)
c.SetRightMargin(0.16)
hBfield.Draw('colz')
c.Draw()

In [ ]:
def varMass(dset, sp1='pi', sp2='K', m1=m_pi, m2=m_K, scalePt=1., shiftE=[0,0], relSmear=0, applyBmap=False):
    c1 = np.ones_like(dset[sp1+'_eta'])
    c2 = np.ones_like(dset[sp2+'_eta'])
    if applyBmap:
        for i, (phi, eta) in enumerate(zip(dset[sp1+'_phi'], dset[sp1+'_eta'])):
            c1[i] = get_bFieldCorr3D(phi, eta)
        print 'Average magnetic field correction: {:1.1e}'.format(np.mean(c1)-1)
        for i, (phi, eta) in enumerate(zip(dset[sp2+'_phi'], dset[sp2+'_eta'])):
            c2[i] = get_bFieldCorr3D(phi, eta)
        
    if relSmear:
        sm1 = relSmear[0]*np.random.randn(dset[sp1+'_pt'].shape[0]) + 1.
        sm2 = relSmear[1]*np.random.randn(dset[sp2+'_pt'].shape[0]) + 1.
    else:
        sm1 = 1.
        sm2 = 1.
    E1 = np.hypot(m1, scalePt * c1 *dset[sp1+'_pt'] * sm1 * np.cosh(dset[sp1+'_eta'])) + shiftE[0]
    E2 = np.hypot(m2, scalePt * c2 *dset[sp2+'_pt'] * sm2 * np.cosh(dset[sp2+'_eta'])) + shiftE[1]
    pt1 = np.sqrt(np.square(E1) - np.square(m1))/np.cosh(dset[sp1+'_eta'])
    pt2 = np.sqrt(np.square(E2) - np.square(m2))/np.cosh(dset[sp2+'_eta'])
    p1p2 = pt1*pt2
    p1p2 *= np.cos(dset[sp1+'_phi'] - dset[sp2+'_phi']) + np.sinh(dset[sp1+'_eta'])*np.sinh(dset[sp2+'_eta'])
    
    M = np.square(m1) + np.square(m2) + 2*(E1*E2 - p1p2)
    return np.sqrt(M)

In [ ]:
def varMass_3(dset, sp1='pi', sp2='K', sp3='pis',
              m1=m_pi, m2=m_K, m3=m_pi,
              scalePt=1., 
              shiftE=[0,0,0], 
              relSmear=0,
              applyBmap=False,
              returnM12=False
             ):
    
    c1 = np.ones_like(dset[sp1+'_eta'])
    c2 = np.ones_like(dset[sp2+'_eta'])
    c3 = np.ones_like(dset[sp2+'_eta'])
    if applyBmap:
        for i, (phi, eta) in enumerate(zip(dset[sp1+'_phi'], dset[sp1+'_eta'])):
            c1[i] = get_bFieldCorr3D(phi, eta)
        print 'Average magnetic field correction: {:1.1e}'.format(np.mean(c1)-1)
        for i, (phi, eta) in enumerate(zip(dset[sp2+'_phi'], dset[sp2+'_eta'])):
            c2[i] = get_bFieldCorr3D(phi, eta)
        for i, (phi, eta) in enumerate(zip(dset[sp3+'_phi'], dset[sp3+'_eta'])):
            c3[i] = get_bFieldCorr3D(phi, eta)
        
    if relSmear:
        sm1 = relSmear[0]*np.random.randn(dset[sp1+'_pt'].shape[0]) + 1.
        sm2 = relSmear[1]*np.random.randn(dset[sp2+'_pt'].shape[0]) + 1.
        sm3 = relSmear[2]*np.random.randn(dset[sp3+'_pt'].shape[0]) + 1.
    else:
        sm1 = 1.
        sm2 = 1.
        sm3 = 1.
        
    E1 = np.hypot(m1, scalePt * c1 * dset[sp1+'_pt'] * sm1 * np.cosh(dset[sp1+'_eta'])) + shiftE[0]
    E2 = np.hypot(m2, scalePt * c2 * dset[sp2+'_pt'] * sm2 * np.cosh(dset[sp2+'_eta'])) + shiftE[1]
    E3 = np.hypot(m3, scalePt * c3 * dset[sp3+'_pt'] * sm3 * np.cosh(dset[sp3+'_eta'])) + shiftE[2]
    
    pt1 = np.sqrt(np.square(E1) - m1**2)/np.cosh(dset[sp1+'_eta'])
    pt2 = np.sqrt(np.square(E2) - m2**2)/np.cosh(dset[sp2+'_eta'])
    pt3 = np.sqrt(np.square(E3) - m3**2)/np.cosh(dset[sp3+'_eta'])
    
    p1p2 = pt1*pt2
    p1p2 *= np.cos(dset[sp1+'_phi'] - dset[sp2+'_phi']) + np.sinh(dset[sp1+'_eta'])*np.sinh(dset[sp2+'_eta'])
    
    p1p3 = pt1*pt3
    p1p3 *= np.cos(dset[sp1+'_phi'] - dset[sp3+'_phi']) + np.sinh(dset[sp1+'_eta'])*np.sinh(dset[sp3+'_eta'])
    
    p2p3 = pt2*pt3
    p2p3 *= np.cos(dset[sp2+'_phi'] - dset[sp3+'_phi']) + np.sinh(dset[sp2+'_eta'])*np.sinh(dset[sp3+'_eta'])
    
    M = np.square(m1) + np.square(m2) + np.square(m3) + 2*(E1*E2 - p1p2) + 2*(E1*E3 - p1p3) + 2*(E2*E3 - p2p3)
    
    if returnM12:
        m12 = np.square(m1) + np.square(m2) + 2*(E1*E2 - p1p2)
        return np.sqrt(M), np.sqrt(m12)
    else:
        return np.sqrt(M)

In [ ]:
def compute_mR(dset, sp1='pi', sp2='K', m1=m_pi, m2=m_K, M=m_D0, scalePt=1., shiftE=[0,0], relSmear=0):
    if relSmear:
        sm1 = relSmear*np.random.randn(dset[sp1+'_pt'].shape[0]) + 1.
        sm2 = relSmear*np.random.randn(dset[sp2+'_pt'].shape[0]) + 1.
    else:
        sm1 = 1.
        sm2 = 1.
    E1 = np.hypot(m1, scalePt * dset[sp1+'_pt'] * sm1 * np.cosh(dset[sp1+'_eta'])) + shiftE[0]
    E2 = np.hypot(m2, scalePt * dset[sp2+'_pt'] * sm2 * np.cosh(dset[sp2+'_eta'])) + shiftE[1]
    mR = np.square(m1)*E2/E1
    mR += np.square(m2)*E1/E2
    mR /= M
    return mR

In [ ]:
def compute_rC(dset, sp1='pi', sp2='K', m1=m_pi, m2=m_K, M=m_D0, scalePt=1., shiftE=[0,0], relSmear=0):
    if relSmear:
        sm1 = relSmear*np.random.randn(dset[sp1+'_pt'].shape[0]) + 1.
        sm2 = relSmear*np.random.randn(dset[sp2+'_pt'].shape[0]) + 1.
    else:
        sm1 = 1.
        sm2 = 1.
    
    q2 = np.square(M) - np.square(m1) - np.square(m2)
    E1 = np.hypot(m1, scalePt * dset[sp1+'_pt'] * sm1 * np.cosh(dset[sp1+'_eta'])) + shiftE[0]
    P1 = np.sqrt(np.square(E1) - np.square(m1))
    E2 = np.hypot(m2, scalePt * dset[sp2+'_pt'] * sm2 * np.cosh(dset[sp2+'_eta'])) + shiftE[1]
    P2 = np.sqrt(np.square(E2) - np.square(m2))
    
    rC_1 = E1*(E2/E1 + 0.5*(q2-2*E1*E2)/np.square(P1))/M
    rC_2 = E2*(E1/E2 + 0.5*(q2-2*E1*E2)/np.square(P2))/M
    return np.column_stack((rC_1, rC_2))

In [ ]:
def getEnergy(dset, sp='pi', m=m_pi, scalePt=1., shiftE=0, relSmear=0):
    if relSmear:
        sm = relSmear*np.random.randn(dset[sp+'_pt'].shape[0]) + 1.
    else:
        sm = 1.
    E = np.hypot(m, scalePt * dset[sp+'_pt'] * sm * np.cosh(dset[sp+'_eta'])) + shiftE
    return E

In [ ]:
def getAvg(mass):
    m = np.mean(mass)
    s = np.std(mass)
    return [m, s/np.sqrt(mass.shape[0])]

# General plots

In [ ]:
def drawVar(varRD, varMC,
            varRDcorr=None, varMCcorr=None,
            binning=None, xtitle='', legLoc=[0.18, 0.7, 0.4, 0.93], opt='overflow+underflow'):
    leg = rt.TLegend(legLoc[0], legLoc[1], legLoc[2], legLoc[3])
    leg.SetBorderSize(0)
    leg.SetFillStyle(0)
    
    hRD = create_TH1D(varRD, 'RDbare',
                      binning=binning, 
                      axis_title=[xtitle, 'Normalized candidates'],
                      scale_histo='norm',
                      opt=opt
                     )
    leg.AddEntry(hRD, 'data', 'lep')

    hMC = create_TH1D(varMC, 'MCbare', binning=hRD.binning, opt=opt, scale_histo='norm')
    hMC.SetLineColor(col_dic['mu'])
    leg.AddEntry(hMC, label_dic['mu'], 'lep')
    h_list = [hRD, hMC]
    
    if not varRDcorr is None:
        hRDc = create_TH1D(varRDcorr, 'RDcorr', binning=hRD.binning, scale_histo='norm', opt=opt)
        hRDc.SetLineColor(1)
        hRD.SetLineColor(rt.kGray)
        h_list.append(hRDc)
        leg.AddEntry(hRDc, 'data corr', 'lep')
        
    if not varMCcorr is None:
        hMCc = create_TH1D(varMCcorr, 'MCcorr', binning=hRD.binning, scale_histo='norm', opt=opt)
        hMCc.SetLineColor(col_dic['mu'])
        hMC.SetLineColor(col_dic['mu']-10)
        h_list.append(hMCc)
        leg.AddEntry(hMCc, label_dic['mu']+' corr', 'lep')
        
    
    SetMaxToMaxHist(h_list)
    c = drawOnCMSCanvas(CMS_lumi, h_list, 'same')
    leg.Draw()
    catText.DrawLatexNDC(0.95, 0.85, 'Category: {}'.format(cat.name))
    c.dnd = h_list + [leg]
    return c

In [ ]:
def draw_dM_vs_x(m_piK_RD, m_piK_MC, m_piK_RDcorr, xRD, xMC, bins=None, xlabel=''):
    avgRD = []
    avgRDcorr = []
    avgMC = []

    for i in range(len(bins)-1):
        m = bins[i]
        M = bins[i+1]   
        selRD = np.logical_and(xRD>m, xRD<M)
        avgRD.append(getAvg(m_piK_RD[selRD]))
        avgRDcorr.append(getAvg(m_piK_RDcorr[selRD]))

        selMC = np.logical_and(xMC>m, xMC<M)
        avgMC.append(getAvg(m_piK_MC[selMC]))

    avgRD = 1e3*np.array(avgRD)
    avgRDcorr = 1e3*np.array(avgRDcorr)
    avgMC = 1e3*np.array(avgMC)

    x = (bins[1:] + bins[:-1])/2.
    dx = (bins[1:] - bins[:-1])/2.

    plt.rcParams.update({'font.size': 18})
    f = plt.figure(figsize=(8,6))
    plt.errorbar(x, avgRD[:,0]-1e3*m_D0, yerr=avgRD[:,1], xerr=dx, fmt='.', label='RD')
    plt.errorbar(x, avgRDcorr[:,0]-1e3*m_D0, yerr=avgRDcorr[:,1], xerr=dx, fmt='.', label='RD corr')
    plt.errorbar(x, avgMC[:,0]-1e3*m_D0, yerr=avgMC[:,1], xerr=dx, fmt='.', label='MC')
    plt.xlabel(xlabel)
    plt.ylabel('<m($\pi$K) - m$_{D_0}$> [MeV]')
    plt.legend(loc='best', numpoints=1)
    plt.grid()

# Corrections

In [ ]:
scalePt=1
shiftE=[0, 0, 0]
# shiftE=[0.48e-3, 0, 0.48e-3]
# shiftE=[0.48e-3, 1.44e-3, 0.48e-3]
relSmear = [3e-3, 3e-3, 6e-3]

# Observables Masses

In [ ]:
m_piK_RDcorr = varMass(df, sp1='pi', sp2='K', m1=m_pi, m2=m_K, scalePt=scalePt, shiftE=shiftE, applyBmap=True)

In [ ]:
m_piK_MCcorr = varMass(df_MC, sp1='pi', sp2='K', m1=m_pi, m2=m_K, relSmear=relSmear)

In [ ]:
C = drawVar(varRD=df['mass_piK'], 
            varMC=df_MC['mass_piK'],
            varRDcorr=m_piK_RDcorr,
            varMCcorr=m_piK_MCcorr,
            binning=[100, m_D0-0.04, m_D0+0.04], 
            xtitle='M(#piK) [GeV]')

In [ ]:
C = drawVar(varRD=df['D0_eta'], 
            varMC=df_MC['D0_eta'],
            binning=[50, -2.2, 2.2], 
            xtitle='D_{0} #eta')

In [ ]:
E = getEnergy(df, 'pi', m_pi, scalePt=scalePt, shiftE=shiftE[0])
ptRD = np.sqrt(np.square(E) - m_pi**2)/np.cosh(df['pi_eta'])
E = getEnergy(df_MC, 'pi', m_pi, relSmear=0)
ptMC = np.sqrt(np.square(E) - m_pi**2)/np.cosh(df_MC['pi_eta'])
C = drawVar(varRD=df['pi_pt'], 
            varMC=df_MC['pi_pt'],
            varRDcorr=ptRD,
            varMCcorr=ptMC,
#             opt='',
            binning=[100, 0, 25], 
            xtitle='#pi p_{T} [GeV]', 
            legLoc=[0.7, 0.6, 0.9, 0.8])

In [ ]:
dsetRDJpsi_loc = '../data/cmsRD/skimmed/B2JpsiKst_200124_{}.root'.format(cat.name)
df_RDjpsi = pd.DataFrame(rtnp.root2array(dsetRDJpsi_loc))
dsetMC_loc = '../data/cmsMC_private/BPH_Tag-Probe_B0_JpsiKst-mumuKpi-kp_13TeV-pythia8_Hardbbbar_PTFilter5_0p0-evtgen_SVV_PU20_10-2-3/'
dsetMC_loc += 'ntuples_B2JpsiKst/skimmed/{}.root'.format(cat.name)
df_MCjpsi= pd.DataFrame(rtnp.root2array(dsetMC_loc))

In [ ]:
C = drawVar(varRD=varMass(df_RDjpsi, sp1='mum', sp2='mup', m1=m_mu, m2=m_mu), 
            varMC=varMass(df_MCjpsi, sp1='mum', sp2='mup', m1=m_mu, m2=m_mu),
            varRDcorr=varMass(df_RDjpsi, sp1='mum', sp2='mup', m1=m_mu, m2=m_mu, scalePt=scalePt, shiftE=shiftE, applyBmap=True),
            varMCcorr=varMass(df_MCjpsi, sp1='mum', sp2='mup', m1=m_mu, m2=m_mu, relSmear=relSmear),
            binning=[50, m_jpsi-0.1, m_jpsi+0.1], 
            xtitle='M(#mu#mu) [GeV]')

In [ ]:
mDst, mD0 = varMass_3(df, sp1='pi', sp2='K', sp3='pis', m1=m_pi, m2=m_K, m3=m_pi, returnM12=True)
mDstMC, mD0MC = varMass_3(df_MC, sp1='pi', sp2='K', sp3='pis', m1=m_pi, m2=m_K, m3=m_pi, returnM12=True)
mDstC, mD0C = varMass_3(df, sp1='pi', sp2='K', sp3='pis', m1=m_pi, m2=m_K, m3=m_pi, returnM12=True, applyBmap=True, scalePt=scalePt, shiftE=shiftE)
mDstMCc, mD0MCc = varMass_3(df_MC, sp1='pi', sp2='K', sp3='pis', m1=m_pi, m2=m_K, m3=m_pi, returnM12=True, relSmear=relSmear)

In [ ]:
mDstMCc, mD0MCc = varMass_3(df_MC, sp1='pi', sp2='K', sp3='pis', 
                            m1=m_pi, m2=m_K, m3=m_pi, returnM12=True, 
                            relSmear=relSmear)

In [ ]:
C = drawVar(varRD=df['mass_D0pis'], 
            varMC=df_MC['mass_D0pis'],
            varRDcorr=mDstC,
            varMCcorr=mDstMCc,
            binning=[100, m_Dst-0.03, m_Dst+0.03], 
            xtitle='M(#piK#pi) [GeV]')

In [ ]:
C = drawVar(varRD=1e3*(mDst - mD0),
            varMC=1e3*(mDstMC - mD0MC),
            varRDcorr=1e3*(mDstC - mD0C),
            varMCcorr=1e3*(mDstMCc - mD0MCc),
            binning=[100, 1e3*dm - 2, 1e3*dm + 2], 
            xtitle='M(#piK#pi) - M(#piK) [MeV]')

### Mass in different regions

In [ ]:
binsPt = np.array([0, 3.2, 5, 8, 12, 20])
avgRD = []
avgRDcorr = []
avgMC = []

for i in range(len(binsPt)-1):
    mpt = binsPt[i]
    Mpt = binsPt[i+1]
    selm = np.logical_and(df['pi_pt']>mpt, df['K_pt']>mpt)
    selM = np.logical_and(df['pi_pt']<Mpt, df['K_pt']<Mpt)    
    selRD = np.logical_and(selm, selM)
    avgRD.append(getAvg(df['mass_piK'][selRD]))
    avgRDcorr.append(getAvg(m_piK_RDcorr[selRD]))
    
    selm = np.logical_and(df_MC['pi_pt']>mpt, df_MC['K_pt']>mpt)
    selM = np.logical_and(df_MC['pi_pt']<Mpt, df_MC['K_pt']<Mpt)    
    selMC = np.logical_and(selm, selM)
    avgMC.append(getAvg(df_MC['mass_piK'][selMC]))

avgRD = 1e3*np.array(avgRD)
avgRDcorr = 1e3*np.array(avgRDcorr)
avgMC = 1e3*np.array(avgMC)

x = (binsPt[1:] + binsPt[:-1])/2.
dx = (binsPt[1:] - binsPt[:-1])/2.
plt.rcParams.update({'font.size': 20})
f = plt.figure(figsize=(8,6))
plt.plot(binsPt, [0]*len(binsPt), 'm--', lw=2)
plt.errorbar(x, avgRD[:,0]-1e3*m_D0, yerr=avgRD[:,1], xerr=dx, fmt='.', label='RD')
plt.errorbar(x, avgRDcorr[:,0]-1e3*m_D0, yerr=avgRDcorr[:,1], xerr=dx, fmt='.', label='RD corr')
plt.errorbar(x, avgMC[:,0]-1e3*m_D0, yerr=avgMC[:,1], xerr=dx, fmt='.', label='MC')
plt.legend(loc='best', numpoints=1)
plt.xlabel('Tracks pT [GeV]')
plt.ylabel('<m($\pi$K) - m$_{D_0}$> [MeV]')

In [ ]:
draw_dM_vs_x(df['mass_piK'], df_MC['mass_piK'], m_piK_RDcorr, df['pi_eta'], df_MC['pi_eta'], 
             np.linspace(-2, 2, 8), r'$\eta_\pi$')

In [ ]:
draw_dM_vs_x(df['mass_piK'], df_MC['mass_piK'], m_piK_RDcorr, df['K_eta'], df_MC['K_eta'], np.linspace(-2, 2, 8), r'$\eta_K$')

In [ ]:
df['D0_phi'].shape[0]

In [ ]:
draw_dM_vs_x(df['mass_piK'], df_MC['mass_piK'], m_piK_RDcorr, df['D0_phi'], df_MC['D0_phi'], np.linspace(-4, 4, 8), r'$D_0\ \phi$ [rad]')

In [ ]:
etaBins = [-2.2, -1, 1., 2.2]
for ie, (em, eM) in enumerate(zip(etaBins[:-1], etaBins[1:])):
    selEtaRD = np.logical_and(df['D0_eta'] > em, df['D0_eta'] < eM)
    selEtaMC = np.logical_and(df_MC['D0_eta'] > em, df_MC['D0_eta'] < eM)
    draw_dM_vs_x(df['mass_piK'][selEtaRD],
                 df_MC['mass_piK'][selEtaMC],
                 m_piK_RDcorr[selEtaRD],
                 df['D0_phi'][selEtaRD], 
                 df_MC['D0_phi'][selEtaMC], 
                 bins=np.linspace(-np.pi, np.pi, 8), 
                 xlabel=r'$D_0\ \phi$ [rad]')
    plt.ylim(plt.ylim()[0], 1.2)
    plt.text(-3.1, 1.05, '{:.1f} < |$\eta$| < {:.1f}'.format(em, eM))

In [ ]:
xRD = compute_mR(df, scalePt=scalePt, shiftE=shiftE)
xMC = compute_mR(df_MC, scalePt=scalePt, shiftE=shiftE)
bins = np.concatenate((np.linspace(0.07, 0.4, 10), np.linspace(0.5, 1.8, 5)))

draw_dM_vs_x(df['mass_piK'], df_MC['mass_piK'], m_piK_RDcorr, xRD, xMC, bins, r'$m_R(D_0\to \pi K)$ [GeV]')

In [ ]:
xMC = compute_rC(df_MC, scalePt=1.+6.6e-4)
xRD = compute_rC(df, scalePt=1.+6.6e-4)

In [ ]:
bins = np.concatenate((np.linspace(0.07, 0.3, 10), np.linspace(0.35, 0.8, 5)))
draw_dM_vs_x(df['mass_piK'], df_MC['mass_piK'], m_piK_RDcorr, xRD[:,0], xMC[:,0], bins, r'$r_C^\pi(D_0\to \pi K)$')

In [ ]:
bins = np.concatenate((np.linspace(0.07, 0.15, 10), np.linspace(0.2, 0.4, 5)))
draw_dM_vs_x(df['mass_piK'], df_MC['mass_piK'], m_piK_RDcorr, xRD[:,0], xMC[:,0], bins, r'$r_C^K(D_0\to \pi K)$')

In [ ]:
binsBG = np.array([0, 10, 15, 20, 30, 50, 70])
bgMC = np.column_stack((df_MC['pi_pt']*np.cosh(df_MC['pi_eta'])/m_pi, df_MC['K_pt']*np.cosh(df_MC['K_eta'])/m_K))
bgRD = np.column_stack((df['pi_pt']*np.cosh(df['pi_eta'])/m_pi, df['K_pt']*np.cosh(df['K_eta'])/m_K))

avgRD = []
avgRDcorr = []
avgMC = []

for i in range(len(binsBG)-1):
    m = binsBG[i]
    M = binsBG[i+1]
    selm = np.logical_and(bgRD[:,0]>m, bgRD[:,1]>m)
    selM = np.logical_and(bgRD[:,0]<M, bgRD[:,1]<M)    
    selRD = np.logical_and(selm, selM)
    avgRD.append(getAvg(df['mass_piK'][selRD]))
    avgRDcorr.append(getAvg(m_piK_RDcorr[selRD]))
    
    selm = np.logical_and(bgMC[:,0]>m, bgMC[:,1]>m)
    selM = np.logical_and(bgMC[:,0]<M, bgMC[:,1]<M)    
    selMC = np.logical_and(selm, selM)
    avgMC.append(getAvg(df_MC['mass_piK'][selMC]))

avgRD = 1e3*np.array(avgRD)
avgRDcorr = 1e3*np.array(avgRDcorr)
avgMC = 1e3*np.array(avgMC)

x = (binsBG[1:] + binsBG[:-1])/2.
dx = (binsBG[1:] - binsBG[:-1])/2.
plt.rcParams.update({'font.size': 20})
f = plt.figure(figsize=(8,6))
plt.errorbar(x, avgRD[:,0]-1e3*m_D0, yerr=avgRD[:,1], xerr=dx, fmt='.', label='RD')
plt.errorbar(x, avgRDcorr[:,0]-1e3*m_D0, yerr=avgRDcorr[:,1], xerr=dx, fmt='.', label='RD corr')
plt.errorbar(x, avgMC[:,0]-1e3*m_D0, yerr=avgMC[:,1], xerr=dx, fmt='.', label='MC')
plt.legend(loc='best', numpoints=1)
plt.xlabel(r'Tracks $\beta \gamma$')
plt.ylabel('<m($\pi$K) - m$_{D_0}$> [MeV]')

In [ ]:
draw_dM_vs_x(df['mass_piK'], df_MC['mass_piK'], m_piK_RDcorr, bgRD[:,0], bgMC[:,0], binsBG, r'$\pi\ \beta \gamma$')

In [ ]:
draw_dM_vs_x(df['mass_piK'], df_MC['mass_piK'], m_piK_RDcorr, bgRD[:,1], bgMC[:,1], binsBG, r'K $\beta \gamma$')

In [ ]:
draw_dM_vs_x(df['mass_piK'], df_MC['mass_piK'], m_piK_RDcorr, df['pi_pt'], df_MC['pi_pt'], binsPt, r'$\pi\ p_T$ [GeV]')

In [ ]:
draw_dM_vs_x(df['mass_piK'], df_MC['mass_piK'], m_piK_RDcorr, df['K_pt'], df_MC['K_pt'], binsPt, r'K $p_T$ [GeV]')